### 基礎設定

In [ ]:
import os
from dotenv import load_dotenv 
load_dotenv(override=True)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

### 記憶管理

In [ ]:
from langchain.messages import HumanMessage, AIMessage, SystemMessage

system_msg = SystemMessage("You are Mego, a bear living in the forest.")
human_msg = HumanMessage("Hello, I'm Luke. Please introduce yourself.")

In [ ]:
messages = [system_msg, human_msg]
messages

In [ ]:
response = llm.invoke(messages)  
response

In [ ]:
messages.extend([AIMessage(content=response.content), HumanMessage(content="What is my name?")])
messages

In [ ]:
response = llm.invoke(messages)  
response.content

### Streaming

In [ ]:
for chunk in llm.stream("你好，好久不見"):
    print(chunk.text, end="|", flush=True)

In [ ]:
full = None
for chunk in llm.stream("你好，好久不見，請介紹下你自己？"):
    full = chunk if full is None else full + chunk
    print(full.text)

### 結構化輸出

In [ ]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

In [ ]:
llm_with_structure = llm.with_structured_output(Movie)
response = llm_with_structure.invoke("Provide details about the movie: Inception")

print(response)
print(type(response))

In [ ]:
llm_with_structure = llm.with_structured_output(Movie, include_raw=True)
response = llm_with_structure.invoke("Provide details about the movie Inception")
response

### 基礎具有短期記憶的對話

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.messages import HumanMessage, AIMessage, SystemMessage
import sys  # 匯入 sys 模組，讓我們可以更精確地控制「即時打字」的效果

# --- 🎨 美化輸出用：定義一些 ANSI 顏色代碼 ---
class Colors:
    """定義終端機顯示的顏色代碼"""
    USER = '\033[96m'      # 亮青色 (使用者)
    AI = '\033[94m'        # 亮藍色 (AI)
    SYSTEM = '\033[93m'    # 亮黃色 (系統訊息)
    RESET = '\033[0m'      # 重置，變回原本的顏色
    BOLD = '\033[1m'       # 粗體

# --- 1️⃣ 準備（初始化）模型 ---
# 這裡是 LangChain 1.0 的標準寫法
print(f"{Colors.SYSTEM}🔄 正在初始化 AI 模型 (gpt-4o-mini)...{Colors.RESET}")
model = ChatOpenAI(
    model="gpt-4o-mini",      # 指定我們要用的 AI 模型 (OpenAI 的聊天模型)
)

# --- 2️⃣ 設定 AI 的「角色」或「系統提示」 (System Prompt) ---
# 告訴 AI 它的「人設」，這會影響它的回答風格
system_message = SystemMessage(
    content="你叫阿明，是一名樂於助人的小幫手。請在對話中，使用台灣口語語氣，並加上表情符號。"
)

# --- 3️⃣ 準備一個「對話紀錄」的列表 (list) ---
# 我們把「人設」當作第一則訊息放進去，讓 AI 隨時記得自己是誰
messages = [system_message]

# --- 顯示歡迎訊息 ---
print(f"{Colors.SYSTEM}{Colors.BOLD}🎉 歡迎來到 AI 聊天室！{Colors.RESET}")
print(f"{Colors.SYSTEM}🔹 你現在在跟「小明」聊天。{Colors.RESET}")
print(f"{Colors.SYSTEM}🔹 輸入 {Colors.BOLD}exit{Colors.RESET}{Colors.SYSTEM} 或 {Colors.BOLD}quit{Colors.RESET}{Colors.SYSTEM} 就可以結束對話。{Colors.RESET}\n")
print(f"{Colors.SYSTEM}{'=' * 50}{Colors.RESET}")


# --- 4️⃣ 進入主要的聊天迴圈 ---
# 支援多輪對話 + 即時打字效果 (streaming)
while True:
    try:
        # --- 步驟 4a: 接收使用者輸入 ---
        # 加上顏色和粗體，讓提示更明顯
        user_input = input(f"{Colors.USER}{Colors.BOLD}👤 你：{Colors.RESET} ")

        print(f"\n{Colors.SYSTEM}{Colors.USER}👤 你：{user_input}{Colors.RESET}")
        
        # --- 步驟 4b: 檢查是否要退出 ---
        if user_input.lower() in {"exit", "quit"}:
            print(f"\n{Colors.SYSTEM}{Colors.BOLD}🧩 收到！對話結束，阿明下班囉，掰掰！{Colors.RESET}")
            break
        
        # --- 步驟 4c: 把使用者剛講的話，加到對話紀錄裡 ---
        messages.append(HumanMessage(content=user_input))

        # --- 步驟 4d: 準備讓 AI 一個字一個字吐出來 ---
        # 先印出 AI 的頭像跟名字，end="" 表示不要換行
        print(f"{Colors.AI}{Colors.BOLD}🤖 阿明：{Colors.RESET}", end="", flush=True)
        
        full_reply = "" # 準備一個空字串，用來存 AI 完整的回答

        # --- 步驟 4e: 呼叫模型，並用「流式輸出」處理 ---
        # 這會像打字機一樣，拿到一點內容就印一點
        for chunk in model.stream(messages):
            # 'chunk' 是一小塊資料，我們把它裡面的 'content' 拿出來
            # 檢查 chunk.content 是否有內容 (有時候會收到空)
            if chunk.content:
                # 直接印出這段小內容
                # 這裡用 print 搭配 end="" 和 flush=True
                # 可以確保內容「馬上」被印出來，不會卡住
                print(f"{Colors.AI}{chunk.content}{Colors.RESET}", end="", flush=True)
                
                # 把這小塊內容，加到 full_reply 裡面存起來
                full_reply += chunk.content

        # AI 講完話了，印一個換行，才不會跟下一行黏在一起
        print() 

        # --- 步驟 4f: 把 AI 剛剛完整的回答，也加到對話紀錄裡 ---
        # 這樣 AI 下一輪才會記得自己講過什麼
        if full_reply: # 確保有收到回覆才新增
            messages.append(AIMessage(content=full_reply))

        # --- 步驟 4g: 保持對話紀錄的長度 ---
        # 為了避免記憶體爆炸，或 API 超出上下文 (Context) 長度限制
        # 我們只保留「系統提示」 + 最近 50 則訊息 (等於 25 輪對話)
        if len(messages) > 51: # 50 則 + 1 則系統提示
            # 保留第一則(人設) + 倒數 50 則
            messages = [messages[0]] + messages[-50:]
        
        # 印出一條漂亮的分隔線，看起來比較舒服
        print(f"{Colors.SYSTEM}{'-' * 50}{Colors.RESET}")

    except KeyboardInterrupt:
        # 處理使用者按 Ctrl+C 的情況 (強制中斷)
        print(f"\n{Colors.SYSTEM}{Colors.BOLD}🖐️ 強制中斷... 再見！{Colors.RESET}")
        break
    except Exception as e:
        # 處理其他所有可能的錯誤
        print(f"\n{Colors.SYSTEM}{Colors.BOLD}🚨 哎呀！出包了：{e}{Colors.RESET}")
        print(f"{Colors.SYSTEM}我們休息一下，請重新啟動程式。{Colors.RESET}")
        break

### 練習

- 多模態的輸入